In [6]:
import Interface as I
from copy import deepcopy
from numpy import array
import matplotlib
import numpy as np
from project_specific_ipynb_code.hot_zone import event_rasterplot, get_doublet_triplet_times_series
import spike_analysis.core
import shutil
import os
import pandas as pd
%matplotlib inline

In [7]:
mdb = I.ModelDataBase('/gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/')
outdir = mdb.create_managed_folder('results', raise_ = False)
network_param_dir = mdb.create_managed_folder('network_param_dir', raise_ = False)

In [3]:
client = I.distributed.Client('10.102.2.81:38786')
client

Client Scheduler: tcp://10.102.2.81:38786 Dashboard: http://10.102.2.81:38787/status,Cluster Workers: 24 Cores: 24 Memory: 824.63 GB


# Copying realistic neuron models to perform simulations

I need realistic neuron models to simulate the L5PTs responses during a particular scenario. To do that I need these files:
- Neuron param. file: ion channel properties
- Hoc: morphology
- Con: map presyn. cells and synapses
- Syn: position of synapses in morphology
- Network param.: activation of presyn. cells and synapses

## Copy hoc files - don't run

In [3]:
mdb_biophysics_py3_BAC2_step = I.ModelDataBase('/gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step')

In [5]:
morphologies = ['89', '90', 'WR69', '88', '93', '84', '87', 'WR64', '85', '86', '91', 'WR71']
for m in morphologies:
    m2 = mdb.create_sub_mdb(m, raise_ = False)    
    target_dir = m2.create_managed_folder('morphology', raise_ = False)
    source_dir = mdb_biophysics_py3_BAC2_step[m]['morphology']
    print('!rsync {} {} -av'.format(source_dir + '/', target_dir + '/'))

!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/89_r7dr3xz7_/mdb/morphology_le6tfw7c_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/89_flfxv4_8_/mdb/morphology_c071my49_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/90_p0yebc2j_/mdb/morphology_316_yebo_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/90_usjdu5xn_/mdb/morphology_qjffp747_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/wr69_4te3iw6b_/mdb/morphology_c8qqxa8i_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/wr69_1nlng4w0_/mdb/morphology_xpprksfz_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/88_3q51cdzv_/mdb/morphology_t6cjtz6h_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/88_tptkwgsi_/mdb/morphology_icvypfl3_/ -av
!rsy

In [6]:
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/89_r7dr3xz7_/mdb/morphology_le6tfw7c_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/89_flfxv4_8_/mdb/morphology_c071my49_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/90_p0yebc2j_/mdb/morphology_316_yebo_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/90_usjdu5xn_/mdb/morphology_qjffp747_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/wr69_4te3iw6b_/mdb/morphology_c8qqxa8i_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/wr69_1nlng4w0_/mdb/morphology_xpprksfz_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/88_3q51cdzv_/mdb/morphology_t6cjtz6h_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/88_tptkwgsi_/mdb/morphology_icvypfl3_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/93_bgmohvju_/mdb/morphology_35pn38kf_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/93_icj4qngu_/mdb/morphology_ihn_h6b1_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/84_fj0n75up_/mdb/morphology_m6aesnuz_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/84_n7wsfi95_/mdb/morphology_yykx_fkz_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/87_4y0gkuui_/mdb/morphology_47b71s4b_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/87_dh6qi30j_/mdb/morphology_538phyh3_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/wr64_5u8jjpef_/mdb/morphology__gxzdriz_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/wr64_xtl8r5ji_/mdb/morphology_xx2o475n_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/85_y_efv5st_/mdb/morphology_cr1_ltei_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/85_7m0u41g2_/mdb/morphology_enh33s1u_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/86_7xx6fpal_/mdb/morphology_w3phro53_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/86_860d25eo_/mdb/morphology_rsqi3_jk_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/91_717kejvg_/mdb/morphology_yv4t2y2g_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/91_2whwg0g7_/mdb/morphology_bu4n8898_/ -av
!rsync /gpfs/soma_fs/scratch/abast/results/20220325_run_low_refractory_period_optimization3_two_BAC_plus_step/wr71_bnihwiry_/mdb/morphology_kkerzn3n_/ /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/wr71_zw1ezeqw_/mdb/morphology_h35e8mwp_/ -av

sending incremental file list
./
89_L5_CDK20050712_nr6L5B_dend_PC_neuron_transform_registered_C2.hoc
Loader.pickle

sent 992,616 bytes  received 57 bytes  1,985,346.00 bytes/sec
total size is 992,123  speedup is 1.00
sending incremental file list
./
90_L5_CDK20050720_nr7L5B_dend_PC_neuron_transform_registered_C2.hoc
Loader.pickle

sent 376,959 bytes  received 57 bytes  754,032.00 bytes/sec
total size is 376,618  speedup is 1.00
sending incremental file list
./
Loader.pickle
WR69_Cell2_L5TT.hoc

sent 530,682 bytes  received 57 bytes  1,061,478.00 bytes/sec
total size is 530,345  speedup is 1.00
sending incremental file list
./
88_L5_CDK20050707_nr5L5B_dend_PC_neuron_transform_registered_C2.hoc
Loader.pickle

sent 361,584 bytes  received 57 bytes  723,282.00 bytes/sec
total size is 361,243  speedup is 1.00
sending incremental file list
./
93_L5_CDK20060808_nr10L5B_dend_PC_neuron_transform_registered_C2.hoc
Loader.pickle

sent 598,476 bytes  received 57 bytes  1,197,066.00 bytes/sec
total

In [13]:
keys_to_copy = ['fixed_params', 'get_fixed_params', 'params', 'get_Simulator', 'get_Evaluator', 'get_Combiner']
for m in morphologies:
    m2 = mdb.create_sub_mdb(m, raise_ = False)    
    for k in keys_to_copy:
        m2[k] = mdb_biophysics_py3_BAC2_step[m][k]

/gpfs/soma_fs/scratch/royo/project_src/in_silico_framework/model_data_base/model_data_base.py:535: UserWarning:

The database source folder has uncommitted changes!

/gpfs/soma_fs/scratch/royo/project_src/in_silico_framework/model_data_base/model_data_base.py:535: UserWarning:

The database source folder has uncommitted changes!

/gpfs/soma_fs/scratch/royo/project_src/in_silico_framework/model_data_base/model_data_base.py:535: UserWarning:

The database source folder has uncommitted changes!

/gpfs/soma_fs/scratch/royo/project_src/in_silico_framework/model_data_base/model_data_base.py:535: UserWarning:

The database source folder has uncommitted changes!

/gpfs/soma_fs/scratch/royo/project_src/in_silico_framework/model_data_base/model_data_base.py:535: UserWarning:

The database source folder has uncommitted changes!

/gpfs/soma_fs/scratch/royo/project_src/in_silico_framework/model_data_base/model_data_base.py:535: UserWarning:

The database source folder has uncommitted changes!

/gpf

## Saving biophysics of a few examplary realistic neuron models in our mdb

In [6]:
mdb_biophyics = I.ModelDataBase('/gpfs/soma_fs/scratch/abast/results/20221020_hot_zone_biophysical_composition')
examplary_models = mdb_biophyics['df_all_selected']
examplary_models.head()
examplary_models = examplary_models[examplary_models['optim_id'] == 'step']


In [16]:
mdb_biophyics['df_all_selected']['morphology_id'].unique()

array(['WR64', 'WR64_Cell8_L5TT_constant_mean_max_dendscale_2.0',
       'WR64_Cell8_L5TT_constant_mean_max_dendscale_2.5', 'WR71',
       'WR69_Cell2_L5TT_mean'], dtype=object)

In [7]:
examplary_models

,description,iloc,value,morphology_id,optim_id,1BAC_APheight,1BAC_ISI,1BAC_ahpdepth,1BAC_caSpike_height,1BAC_caSpike_width,...,BAC_ahpdepth,BAC_caSpike_height,BAC_caSpike_width,BAC_spikecount,ephys.CaDynamics_E2.apic.decay,ephys.CaDynamics_E2.apic.gamma,ephys.CaDynamics_E2.axon.decay,ephys.CaDynamics_E2.axon.gamma,ephys.CaDynamics_E2.soma.decay,ephys.CaDynamics_E2.soma.gamma
0,max_depo_Ca_HVA,547257,0.991423,WR64,step,0.711611,2.936480,0.334893,2.999421,2.594488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,max_depo_Ca_LVAst,807892,0.961259,WR64,step,1.225208,1.305037,0.258493,2.986296,0.929134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,max_depo_NaTa_t,811265,0.052416,WR64,step,0.926502,0.191969,0.177453,2.018352,0.862205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,max_hyperpo_SK_E2,751477,0.999981,WR64,step,0.909364,1.762358,0.391443,0.836715,1.078740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,max_hyperpo_Im,850424,0.999472,WR64,step,0.823408,2.261360,0.284071,0.952462,2.791339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,max_hyperpo_SKv3_1,832799,0.980978,WR64,step,0.915606,0.558882,0.282351,1.347697,0.389764,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,max_apical_thickness,653684,3.000000,WR64,step,1.124353,0.720324,0.386223,0.504938,1.216535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,min_apical_thickness,705467,2.677242,WR64,step,1.122521,1.410121,0.258115,1.768310,2.515748,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,best_model_seed_10987281838369298,479057,3.673609,WR64,step,0.890802,0.749677,0.100356,3.054818,2.259843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,best_model_seed_20007698105833676,480017,2.734112,WR64,step,1.049423,1.835740,0.399196,2.723234,0.043307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
mdb['examplary_models'] = examplary_models

/gpfs/soma_fs/scratch/royo/project_src/in_silico_framework/model_data_base/model_data_base.py:535: UserWarning:

The database source folder has uncommitted changes!



# Biophysical properties of the cell

We have different reconstructed morphologies. Each morphology can have a set of different biophysical parameters, resulting all of the different parameter combinations in realistic behavior of the cell. These biophysical properties have been found with either a biophysical parameter fitting algorithm (genetic alg.) or parameter exploration one. Here we look at just a few examples.
During this step we create a neuron param file for a examplary model.

In [9]:
params_py2 = ['ephys.CaDynamics_E2.apic.decay',
       'ephys.CaDynamics_E2.apic.gamma', 'ephys.CaDynamics_E2.axon.decay',
       'ephys.CaDynamics_E2.axon.gamma', 'ephys.CaDynamics_E2.soma.decay',
       'ephys.CaDynamics_E2.soma.gamma', 'ephys.Ca_HVA.apic.gCa_HVAbar',
       'ephys.Ca_HVA.axon.gCa_HVAbar', 'ephys.Ca_HVA.soma.gCa_HVAbar',
       'ephys.Ca_LVAst.apic.gCa_LVAstbar', 'ephys.Ca_LVAst.axon.gCa_LVAstbar',
       'ephys.Ca_LVAst.soma.gCa_LVAstbar', 'ephys.Im.apic.gImbar',
       'ephys.K_Pst.axon.gK_Pstbar', 'ephys.K_Pst.soma.gK_Pstbar',
       'ephys.K_Tst.axon.gK_Tstbar', 'ephys.K_Tst.soma.gK_Tstbar',
       'ephys.NaTa_t.apic.gNaTa_tbar', 'ephys.NaTa_t.axon.gNaTa_tbar',
       'ephys.NaTa_t.soma.gNaTa_tbar', 'ephys.Nap_Et2.axon.gNap_Et2bar',
       'ephys.Nap_Et2.soma.gNap_Et2bar', 'ephys.SK_E2.apic.gSK_E2bar',
       'ephys.SK_E2.axon.gSK_E2bar', 'ephys.SK_E2.soma.gSK_E2bar',
       'ephys.SKv3_1.apic.gSKv3_1bar', 'ephys.SKv3_1.apic.offset',
       'ephys.SKv3_1.apic.slope', 'ephys.SKv3_1.axon.gSKv3_1bar',
       'ephys.SKv3_1.soma.gSKv3_1bar', 'ephys.none.apic.g_pas',
       'ephys.none.axon.g_pas', 'ephys.none.dend.g_pas',
       'ephys.none.soma.g_pas', 'scale_apical.scale']
params_py3 = [p.replace('ephys.CaDynamics_E2', 'ephys.CaDynamics_E2_v2') for p in params_py2]

Get the biophysical parameters of one examplary biophysical model

In [6]:
examplary_models = mdb['examplary_models']
bio_params = examplary_models.iloc[0][params_py3]
print(bio_params)

ephys.CaDynamics_E2_v2.apic.decay        55.1313
ephys.CaDynamics_E2_v2.apic.gamma      0.0005357
ephys.CaDynamics_E2_v2.axon.decay        143.024
ephys.CaDynamics_E2_v2.axon.gamma     0.00445041
ephys.CaDynamics_E2_v2.soma.decay        668.442
ephys.CaDynamics_E2_v2.soma.gamma     0.00549685
ephys.Ca_HVA.apic.gCa_HVAbar          0.00489763
ephys.Ca_HVA.axon.gCa_HVAbar         0.000584527
ephys.Ca_HVA.soma.gCa_HVAbar         2.59393e-05
ephys.Ca_LVAst.apic.gCa_LVAstbar      0.00035929
ephys.Ca_LVAst.axon.gCa_LVAstbar     0.000244502
ephys.Ca_LVAst.soma.gCa_LVAstbar     0.000190162
ephys.Im.apic.gImbar                 0.000564525
ephys.K_Pst.axon.gK_Pstbar             0.0879311
ephys.K_Pst.soma.gK_Pstbar             0.0351482
ephys.K_Tst.axon.gK_Tstbar               0.02997
ephys.K_Tst.soma.gK_Tstbar             0.0735116
ephys.NaTa_t.apic.gNaTa_tbar           0.0161037
ephys.NaTa_t.axon.gNaTa_tbar             1.31117
ephys.NaTa_t.soma.gNaTa_tbar             1.97784
ephys.Nap_Et2.axon.g

### Build a neuron param file for that examplary model.

The biophysics_fitting module has the functionality to build a neuron param file: the get_Simulator (cell_param_generator or get_cell_params_with_default_sim_prams methods) object can be used. I could also create it manually (making sure it is a sumatra object: I.scp.NTParameterSet(dictionary))

In [7]:
def get_simulator(id_, py2 = False):
    s = mdb[id_]['get_Simulator'](mdb[id_])
    return s

# necessary for creating the neuron param file
s = get_simulator('WR64')
neup = s.setup.get_cell_params_with_default_sim_prams(bio_params)
neup

{'neuron': {'AIS': {'mechanisms': {'global': {},
    'range': {'CaDynamics_E2_v2': {'decay': 143.02406190141963,
      'gamma': 0.0044504149834418095,
      'spatial': 'uniform'},
     'Ca_HVA': {'gCa_HVAbar': 0.0005845265134372266, 'spatial': 'uniform'},
     'Ca_LVAst': {'gCa_LVAstbar': 0.0002445016694479747, 'spatial': 'uniform'},
     'Ih': {'gIhbar': 8e-05, 'spatial': 'uniform'},
     'K_Pst': {'gK_Pstbar': 0.0879310708336956, 'spatial': 'uniform'},
     'K_Tst': {'gK_Tstbar': 0.029970047681400293, 'spatial': 'uniform'},
     'NaTa_t': {'gNaTa_tbar': 1.3111744676343984, 'spatial': 'uniform'},
     'Nap_Et2': {'gNap_Et2bar': 0.0059599487687635394, 'spatial': 'uniform'},
     'SK_E2': {'gSK_E2bar': 0.037368286778582255, 'spatial': 'uniform'},
     'SKv3_1': {'gSKv3_1bar': 1.2372469631121323, 'spatial': 'uniform'},
     'pas': {'e': -90, 'g': 2.404469610465027e-05, 'spatial': 'uniform'}}},
   'properties': {'Ra': 100.0, 'cm': 1.0, 'ions': {'ek': -85.0, 'ena': 50.0}}},
  'ApicalDendri

The path below shows where the cell morphology file for that model is saved.

In [11]:
neup.neuron.filename

'/gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/wr64_xtl8r5ji_/mdb/morphology_xx2o475n_/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean.hoc'

In [8]:
n_models = len(examplary_models)
print('We have {} examplary models'.format(n_models))

#for i in range(n_models):
#    p = df.iloc[i][params_py3]
#    morph = df.iloc[i]['morphology_id']
#    s = get_simulator(morph)
#    neup = s.setup.get_cell_params_with_default_sim_prams(p)
#    print(neup)

We have 32 examplary models


# Network embedding
Here, we place a cell in barrel cortex and compute an anatomical realization, which specifies where synapses are located and their properties. Afterwards, we activate the synapses depending on the scenario we would like to simulate.

## Step 1: Register cell morphology in the BC reference frame and calculate positions of synapses

When a neuron is reconstructed, it's morphology is saved in a .hoc file. Such morphology needs to be placed in the barrel cortex (BC) and written in terms of the BC reference frame. Depending on where the neuron lies in the BC, it's morphology is slightly adapted. To do this external resources are used (Cortex In Silico website / NeuroMorph pipeline, check /Share/HowTos/NeuroMapDocumentation), which provide us with a registered .hoc morphology. (Most of) the morphologies we current work with have previously been registered.

We then use the I.map_singlecell_inputs to calculate the position of synapses onto the cell, what we also refer as 'anatomical model'. Under the hood, 50 anatomical realizations will be computed. Out of these, the one that is closest to the average is choosen, which then can be refered to as "representative realization". The result is saved in the same folder as the hoc morphology. This takes about 4 hrs to compute.
During this step the .con and .syn files get generated.

In the past, for building the anatomical model we needed to use the morphology file with scaled apical trunk. Now that's not necessary as the diameters will be scaled later with neuron. We specify the scaling factor in the neuron parameter file itself, which wasn't possible at the time. 

Specifiying the location of synapses does not depend on the diameter of the trunk. It would depend on the diameter for INH synapses as they are dependant on the surface area. However, the diameter fitting is for fitting cellular properties. The hoc file for the network embedding needs to be the unscaled one. 

In [12]:
# Building anatomical model
with I.silence_stdout: # this creates > 1 GB of diagnostic information
    I.map_singlecell_inputs(neup.neuron.filename, 'L5tt') # this takes some time

Connection dropped: outstanding heartbeat ping not received
Transition to CONNECTING
Connection dropped: outstanding heartbeat ping not received
Transition to CONNECTING
Connection dropped: outstanding heartbeat ping not received
Transition to CONNECTING
Connection dropped: outstanding heartbeat ping not received
Transition to CONNECTING


Now copy the results in my mdb. Right now they are in the same location as the morphology .hoc file.

In [10]:
!rsync /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/wr64_xtl8r5ji_/mdb/morphology_xx2o475n_/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492 /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/network_param_dir_6kz2949w_/ -av
!rm -rf /gpfs/soma_fs/scratch/royo/results/20230621_Whisking_v3/wr64_xtl8r5ji_/mdb/morphology_xx2o475n_/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492

sending incremental file list
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/NumberOfConnectedCells.csv
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_summary_20230711-1128_182492.csv
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492.con
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492.syn
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cel

WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L2_B3_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L2_B4_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L2_Beta_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L2_C1_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell

WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L4sp_C3_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L4sp_C4_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L4sp_D1_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L4sp_D2_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR6

WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L5tt_C3_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L5tt_C4_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L5tt_D1_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_L5tt_D2_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR6

WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_SymLocal4_A4_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_SymLocal4_Alpha_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_SymLocal4_B1_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/apical_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_apical_synapses_SymLocal4_B2_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182

...

WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L2_B2_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L2_B3_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L2_B4_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L2_Beta_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2

WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L4sp_C3_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L4sp_C4_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L4sp_D1_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L4sp_D2_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L

WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L5st_C4_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L5st_D1_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L5st_D2_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L5st_D3_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L

WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L6ccinv_C3_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L6ccinv_C4_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L6ccinv_D1_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_total_synapses_L6ccinv_D2_20230711-1128_182492.landmarkAscii
WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492/total_synapses/


sent 2,997,043 bytes  received 40,272 bytes  2,024,876.67 bytes/sec
total size is 2,788,684  speedup is 0.92


In [18]:
anatomical_model_dir = os.path.join(mdb['network_param_dir'],'WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492')
syn_file = os.path.join(registered_morph_dir,'WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492.syn')
con_file = os.path.join(registered_morph_dir,'WR64_Cell8_L5TT_C2-registered.hocwith_radius_constant_mean_synapses_20230711-1128_182492.con')

## Step 2: Define synapse activation

Now we need to activate the synapses according to the scenario we want to reproduce (passive/active touch, etc.).
To do this we need the following information:
- The characteristics of the synapses by celltype: 
    - Synapse strength: estimated by activating presyn cells one by one and measuring the deflection at the soma. Strength is set such that the uEPSP distribution matches empirical measurements. Code to this this: project_src/in_silico_framework/simrun3/synaptic_strength_fitting.py, example how to do this: http://10.102.3.18:11112/notebooks/notebooks_backup_nas1_20221021/20190213_reproducing_L6paper_control_with_CDK91_morphology_model.ipynb


- The activity by celltype during that scenario 

Note: the cell below has not been run yet.

In [ ]:
# Synapse strength
psp = PSPs(cell_param, confile) # generate PSP object, it needs to know the cell parameters and the connections
psp.run(c) # runt the simulations with dask, c is a client object
psp.get_voltage_traces() # look at simulated voltage traces
psp.get_optimal_g(I.barrel_cortex.get_EPSP_measurement() # get fitted strengths given the EPSP measurements from barrel cortex

The following results have been copied from other notebook, change accordingly. This information is used to create the syn, con and network param files.

In [ ]:
# http://10.102.3.85:11112/notebooks/20220622_hot_zone_simulation_WR64_const_PSTH_randomly_distributed_inh_RN_MG_dataset_low_rp_models_20ms_onset_4_science_paper.ipynb
syn_strength_dict_all_synapses = {'2020-01-14_99896_D2P2qpH_81_48_362': {'L2': 1.7833905609143366,
  'L34': 1.4375495495904855,
  'L4': 1.217763010150186,
  'L5st': 1.5480934081344324,
  'L5tt': 1.3759639995806972,
  'L6cc': 1.3214324732847653,
  'L6ct': 1.5129106825731375,
  'VPM': 0.2722515945386025},
 '2020-01-14_99896_D2P2qpH_82_100_609': {'L2': 1.9582070655085562,
  'L34': 1.471556288556276,
  'L4': 1.2314818975716677,
  'L5st': 1.6714027571337364,
  'L5tt': 1.3823607473439572,
  'L6cc': 1.3301471360411359,
  'L6ct': 1.5132357766140985,
  'VPM': 0.20312625153875144},
 '2020-01-14_99896_D2P2qpH_83_202_310': {'L2': 1.9632505535764149,
  'L34': 1.4792745789496118,
  'L4': 1.2360082985949101,
  'L5st': 1.6688720591078823,
  'L5tt': 1.3936349514991342,
  'L6cc': 1.340946466492963,
  'L6ct': 1.5266717051164975,
  'VPM': 0.23784867892328862},
 '2020-01-14_99896_D2P2qpH_84_544_372': {'L2': 1.8218926353644682,
  'L34': 1.4536439107326744,
  'L4': 1.226891199638447,
  'L5st': 1.5851439663741975,
  'L5tt': 1.382944229681059,
  'L6cc': 1.329449273607428,
  'L6ct': 1.5184184803961167,
  'VPM': 0.23126226568765898},
 '2022-03-25_173607_HnObcej_10987281838369298_383_722': {'L2': 2.07850514385837,
  'L34': 1.6879020616645104,
  'L4': 1.4180445425420305,
  'L5st': 1.8347113759927205,
  'L5tt': 1.6293281369495252,
  'L6cc': 1.5715156552393867,
  'L6ct': 1.7681532348752989,
  'VPM': 0.42912071914774835},
 '2022-03-25_173607_HnObcej_215286051698869_76_278': {'L2': 2.120256508532983,
  'L34': 1.6991419670007928,
  'L4': 1.424767185952692,
  'L5st': 1.8575796112052565,
  'L5tt': 1.6352503458772545,
  'L6cc': 1.5785270534447406,
  'L6ct': 1.772573069756413,
  'VPM': 0.4258908527416482},
 '2022-03-25_173607_HnObcej_47168304239529988_292_71': {'L2': 2.1763193229750377,
  'L34': 1.7169721020650854,
  'L4': 1.4320370404009988,
  'L5st': 1.9074885745135286,
  'L5tt': 1.653981499337697,
  'L6cc': 1.5951555624242464,
  'L6ct': 1.7802283542272501,
  'VPM': 0.4320477484932132},
 '2022-03-25_173607_HnObcej_51186661787927645_280_123': {'L2': 2.440366454619788,
  'L34': 1.782228798236008,
  'L4': 1.4721227603006022,
  'L5st': 2.094347208688859,
  'L5tt': 1.7035468403823106,
  'L6cc': 1.643185486466752,
  'L6ct': 1.8217001459425806,
  'VPM': 0.4293361464572546},
 '2022-03-25_173607_HnObcej_58927627030497273_468_34': {'L2': 2.139847202301498,
  'L34': 1.7021734134771678,
  'L4': 1.423119252710654,
  'L5st': 1.8717591811690248,
  'L5tt': 1.6387231137196632,
  'L6cc': 1.5818807657691787,
  'L6ct': 1.769806196677751,
  'VPM': 0.43351516845646626},
 '2022-03-25_173607_HnObcej_75398750937823036_433_493': {'L2': 2.2282802177840617,
  'L34': 1.736031173102915,
  'L4': 1.4461370524841548,
  'L5st': 1.9506985344298244,
  'L5tt': 1.6714730335610746,
  'L6cc': 1.6127076711858894,
  'L6ct': 1.798249840162311,
  'VPM': 0.4408098317707232},
 '2022-03-25_173607_HnObcej_84342990166712235_337_539': {'L2': 2.146342762606754,
  'L34': 1.7078142596796664,
  'L4': 1.4281618380657124,
  'L5st': 1.8775705194503032,
  'L5tt': 1.6462063363213026,
  'L6cc': 1.5872844355992304,
  'L6ct': 1.7787651867053984,
  'VPM': 0.43390532844006896}}

template_EXC = {'cellNr': None,
 'celltype': {'pointcell': {'distribution': 'PSTH_poissontrain_v2',
   'intervals': None,#None
   'offset': 0.0,
   'rates': None}},
 'synapses': {'connectionFile': None,
  'distributionFile': None,
  'receptors': {'glutamate_syn': {'delay': 0.0,
    'parameter': {'decayampa': 1.0,
     'decaynmda': 1.0,
     'facilampa': 0.0,
     'facilnmda': 0.0,
     'tau1': 26.0,
     'tau2': 2.0,
     'tau3': 2.0,
     'tau4': 0.1},
    'threshold': 0.0,
    'weight': [None, None]}},
  'releaseProb': 0.6}}

template_INH= {'cellNr': None,
 'celltype': {'pointcell': {'distribution': 'PSTH_poissontrain_v2',
   'intervals': None,#None
   'offset': 0.0,
   'rates': None}},
 'synapses': {'connectionFile': None,
  'distributionFile': None,
  'receptors': {'gaba_syn': {'delay': 0.0,
    'parameter': {'decaygaba': 1.0,
     'decaytime': 20.0,
     'e': -80.0,
     'facilgaba': 0.0,
     'risetime': 1.0},
    'threshold': 0.0,
    'weight': 1.0}},
  'releaseProb': 0.25}}

template_info = {'author': 'abast',
 'date': '23Sep2021',
 'name': 'asd'}

template_NMODL_mechanisms = {'VecStim': '/', 'synapses': '/'}

# Perform simulations